In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np

In [10]:
from models import Simple_NN, Trainer_GCN
from dataset_graphs import graph_from_design, populate_node_x
from dataset_traindata import ClassificationData

In [11]:
# Constants
SOURCE = "../raw/dataset_1/"

## Training a simple NN

In [12]:
# Hyperparameters
num_epochs = 250
batch_size = 32

In [13]:
# Data loading from csv
dataset = ClassificationData(SOURCE)
train_dataloader = DataLoader(dataset, batch_size=batch_size)

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

Using cuda


In [15]:
model = Simple_NN(input_size=4, output_size=2).to(device)

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 5 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [18]:
for t in range(num_epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Done!


## Predicting NN using Trainer GNN

In [22]:
# Preparing data
features = np.genfromtxt(SOURCE + "data_features.csv", delimiter=', ')
targets = np.genfromtxt(SOURCE + "data_targets.csv", delimiter=',')
design = [4, 64, 64, 2]
data = graph_from_design(design)
populate_node_x(data, design, features, targets)

In [23]:
data

Data(design=[4], num_nodes=134, edge_index=[2, 8960], x=[134, 500])

In [24]:
# Loading model
model = Trainer_GCN()
model.load_state_dict(torch.load("../model/model"))
model.eval()

Trainer_GCN(
  (conv1): GCNConv(500, 256)
  (conv2): GCNConv(256, 64)
  (denseB): Linear(in_features=64, out_features=1, bias=True)
  (denseW): Linear(in_features=64, out_features=1, bias=True)
)

In [25]:
# Process data
out = model(data)

## Model Comparison

In [ ]:
data = dataset[0:10]

In [ ]:
input, output = data
input = input.to(device)
output = output.to(device)

In [ ]:
out = model(input)
print("Real: Predicted:")
for y, y_pred in zip(out, output):
    print(y.argmax().item(), y_pred.argmax().item())

Real: Predicted:
1 1
0 0
1 1
1 1
1 1
1 1
0 0
1 1
1 1
0 0
